Demonstrations for context <a class="ProveItLink" href="_context_.ipynb">proveit.logic.equality</a>
========

In [1]:
import proveit
%begin demonstrations

## Substitution

Equality is a fundamental concept of logic.  When two mathematical operations are equal, then one may be substituted for the other.  This is rule is defined by the `substitution` axiom:

In [2]:
from proveit.logic.equality._axioms_ import substitution
from proveit._common_ import a, b, c, d, x, y, z, fx # we'll use these later
substitution

|= forall_{f, x, y | x = y} (f(x) = f(y))

The English translation of this axiom is: for any function $f$ and any $x, y$ such that $x=y$, $f(x) = f(y)$.  In other words, we may substitute $y$ for $x$ in any function whenever $x=y$.  The equality of $x$ and $y$ transfers to an equality between $f(x)$ and $f(y)$.  This is fundamental to the meaning of equality regardless of what $f$ does (as long as it can act on a single argument).  We may specialize this axiom using any operation for $f$.  For example,

In [3]:
from proveit.logic import Not, Equals
substitution.specialize({fx:Not(x), x:a, y:b}, assumptions=[Equals(a, b)])

{a = b} |= [not](a) = [not](b)

There are more convenient ways to apply this substitution rule than manual specialization that was demonstrated in the previous input.  The `Equals` class has the `substitution`, `subLeftSideInto` and `subRightSideInto` methods for conveniently applying substitution and its variants, as we will demonstrate below.  Each of these methods takes a `lambdaMap` argument to provide a context for the substitution -- what is being substituted and where.  A `lambdaMap` can be an actual **Lambda** expression, an **InnerExpr** object, or if it is neither of these, it can be any other **Expression** for which the default will be to performa a global replacement.

In [4]:
from proveit import Lambda
from proveit.number import Add, Frac, Exp
expr = Equals(a, Add(b, Frac(c, d), Exp(c, d)))          

expr: a = (b + (c / d) + c^{d})

The `globalRepl` static method of `Lambda` is useful for creating a global replacement lambda map.  Below, we create a map for replacing every occurence of $d$ in `expr` with anything else:

In [5]:
gRepl = Lambda.globalRepl(expr, d)

gRepl: _x_ -> (a = (b + (c / _x_) + c^{_x_}))

We now use this lambda map to replace occurences of $d$ in `expr` with $y$:

In [6]:
d_eq_y = Equals(d, y)

d_eq_y: d = y

In [7]:
d_eq_y.substitution(gRepl, assumptions=[d_eq_y])

{d = y} |= (a = (b + (c / d) + c^{d})) = (a = (b + (c / y) + c^{y}))

Or we can take advantage of that a global replacement is performed by default when a non-**Lambda** expression is provided as the "`lambdaMap`".

In [8]:
d_eq_y.substitution(expr, assumptions=[d_eq_y])

{d = y} |= (a = (b + (c / d) + c^{d})) = (a = (b + (c / y) + c^{y}))

Either way, the generated proof is the same:

In [9]:
d_eq_y.substitution(expr, assumptions=[d_eq_y]).proof()

step type	requirements	statement
0	specialization	1, 2	{d = y} |= (a = (b + (c / d) + c^{d})) = (a = (b + (c / y) + c^{y}))
	{f -> [_x_ -> (a = (b + (c / _x_) + c^{_x_}))]}, {x -> d}, {y -> y}
1	axiom		|= forall_{f, x, y | x = y} (f(x) = f(y))
	proveit.logic.equality.substitution
2	assumption		{d = y} |= d = y

If we want to perform substitution for a specific inner expression, and not necessarily a global replacement, the `proveit._core_.expression.InnerExpr` class (aliased as `proveit.InnerExpr`) is extremely convenient.  It uses some Python tricks via implementing the `__getitem__` and `__getattr__` methods.  First, you create the `InnerExpr` object by calling the `innerExpr()` method on the top-level expression:

In [10]:
innerExpr = expr.innerExpr()

innerExpr: {lambda:[_x_ -> _x_], _x_:(a = (b + (c / d) + c^{d}))}

The `InnerExpr` object displays itself with two important pieces of information: the lambda map that it represents and the inner expression of the top-level expression that would be replaced by this lambda map.  The point is that we will be able to "dig" in to inner expressions of the top-level expression via accessing sub-expression attributes.  For example, we next will "dig" into the "right hand side" (`rhs`) of the master expression: 

In [11]:
innerExpr = innerExpr.rhs

innerExpr: {lambda:[_x_ -> (a = _x_)], _x_:(b + (c / d) + c^{d})}

By accessing the `rhs` attribute, we created a new `InnerExpr` object that has the same top-level expression as the original but has a new current inner expression.  Note that the `InnerExpr` class does not know anything about the `rhs` attribute itself; it is relying on the fact that the previous sub-expression has this attribute.  The `InnerExpr` class also has tricks for getting an sub-expression with an index (or key):

In [12]:
innerExpr = innerExpr.operands[1]

innerExpr: {lambda:[_x_ -> (a = (b + _x_ + c^{d}))], _x_:(c / d)}

Now will "dig" down to the denominator of $\frac{c}{d}$ and show how we use the `InnerExpr` class to replace a particular occurence of $d$ rather than a global replacement:

In [13]:
innerExpr = innerExpr.denominator

innerExpr: {lambda:[_x_ -> (a = (b + (c / _x_) + c^{d}))], _x_:d}

In [14]:
d_eq_y.substitution(innerExpr, assumptions=[d_eq_y])

{d = y} |= (a = (b + (c / d) + c^{d})) = (a = (b + (c / y) + c^{d}))

Let us demonstrate this technique again, replacing the other occurrence of $d$.  This time we do this more succinctly, without the extra pedogogial steps:

In [15]:
d_eq_y.substitution(expr.innerExpr().rhs.operands[2].exponent, assumptions=[d_eq_y])

{d = y} |= (a = (b + (c / d) + c^{d})) = (a = (b + (c / d) + c^{y}))

The `substition` method, that we demonstrated above, is a direct application of the `substitution` axiom.  It proves the equality between some $f(x)$ and some $f(y)$.  We often will want to take a shortcut to perform a statement substitution in which we prove some $P(y)$ is true assuming that $P(x)$ is true and $P(x) = P(y)$.  For this, we have the `subRightSideInto` and `subLeftSideInto` methods.

If the expression that we want to substitute in is on the right hand side of the `Equals` object playing the role of $x=y$, then we use `subRightSideInto`: 

In [16]:
d_eq_y.subRightSideInto(gRepl, assumptions=[d_eq_y,expr])

{d = y , a = (b + (c / d) + c^{d})} |= a = (b + (c / y) + c^{y})

We can also take advantage of the global replacement default and provide a non-**Lambda** **Expression**.

In [17]:
d_eq_y.subRightSideInto(expr, assumptions=[d_eq_y,expr])

{d = y , a = (b + (c / d) + c^{d})} |= a = (b + (c / y) + c^{y})

If the expression that we want to substitute in is on the left hand side of the `Equals` object playing the role of $x=y$, then we use `subLeftSideInto`: 

In [18]:
y_eq_d = Equals(y, d)

y_eq_d: y = d

In [19]:
y_eq_d.subLeftSideInto(gRepl, assumptions=[y_eq_d,expr])

{y = d , a = (b + (c / d) + c^{d})} |= a = (b + (c / y) + c^{y})

Again, we can provide a non-**Lambda** **Expression** to do a simple global replacement.

In [20]:
y_eq_d.subLeftSideInto(expr, assumptions=[y_eq_d,expr])

{y = d , a = (b + (c / d) + c^{d})} |= a = (b + (c / y) + c^{y})

The proof uses a theorem the relies upon the `substitution` axiom, rather than using the `substition` axiom directly:

In [21]:
y_eq_d.subLeftSideInto(expr, assumptions=[y_eq_d,expr]).proof()

step type	requirements	statement
0	specialization	1, 2, 3	{y = d , a = (b + (c / d) + c^{d})} |= a = (b + (c / y) + c^{y})
	{P -> [_x_ -> (a = (b + (c / _x_) + c^{_x_}))]}, {x -> y}, {y -> d}
1	theorem		|= forall_{P, x, y | P(y) , x = y} P(x)
	proveit.logic.equality.subLeftSideInto
2	assumption		{a = (b + (c / d) + c^{d})} |= a = (b + (c / d) + c^{d})
3	assumption		{y = d} |= y = d

## Reflexivity, symmetry, and transitivity

Reflexivity, symmetry, and transitivity are also fundamental properties of equality, in addition to the ability to perform substitution.  Reflexivity is the fact that any mathematical object is equal to itself.  Symmetry is the fact that $x = y$ and $y = x$ are equivalent (either both of these are true or both of these false).  Transitivity is the ability to derive $x=z$ from $x=y$ and $y=z$.  These are all axioms.

In [22]:
from proveit.logic.equality._axioms_ import equalsReflexivity, equalsSymmetry, equalsTransitivity

In [23]:
equalsReflexivity

|= forall_{x} (x = x)

In [24]:
equalsSymmetry

|= forall_{x, y} ((y = x) = (x = y))

In [25]:
equalsTransitivity

|= forall_{x, y, z | x = y , y = z} (x = z)

`equalsReversal` is a useful theorem for applying the symmetry property of equality:

In [26]:
from proveit.logic.equality._theorems_ import equalsReversal
equalsReversal # y=x derives from x=y

|= forall_{x, y | x = y} (y = x)

These three properties are applied automatically for `Equals` objects

Reflexivity is concluded automatically:

In [27]:
Equals(a, a).prove()

|= a = a

In [28]:
Equals(a, a).prove().proof()

step type	requirements	statement
0	specialization	1	|= a = a
	x -> a
1	axiom		|= forall_{x} (x = x)
	proveit.logic.equality.equalsReflexivity

Symmetric statements are derived as side-effects.  Note that the `KnownTruth.deriveSideEffects` method employs a mechanism to prevent infinite recursion or this would not be possible (it would continually go back and forth, proving $y=x$ from $x=y$ then $x=y$ from $y=x$, ad infinitum)

In [29]:
a_eq_b = Equals(a, b)

a_eq_b: a = b

In [30]:
Equals(b, a).prove([a_eq_b])

{a = b} |= b = a

The reversed form may also be derived explicitly via `deriveReversed`.  The proof is the same.

In [31]:
a_eq_b.prove([a_eq_b]).deriveReversed().proof()

step type	requirements	statement
0	specialization	1, 2	{a = b} |= b = a
	{x -> a}, {y -> b}
1	theorem		|= forall_{x, y | x = y} (y = x)
	proveit.logic.equality.equalsReversal
2	assumption		{a = b} |= a = b

Transitivity derivations are attempted with automation via the `conclude` method using the `trensitivitySearch` function in `proveit.logic.equality.transitivity_search`.  This performs a breadth-first, bidirectional search (meeting in the middle from both ends) over the space of **KnownTruth** objects representing equality and using appropriate assumptions.  This is therefore reasonably efficient.  Efficiency should not really be an issue, anyways, as long as proofs for each theorem are relatively small.  A long proof should be broken up into several smaller proofs for lemma-like theorems.  In that case (in the setting of small proofs), the space of **KnownTruth**s will be small and this search algorithm will have ample efficiency. 

In [32]:
# We'll make this interesting by reversing some of the equations in the chain.
c_eq_b = Equals(c, b) 

c_eq_b: c = b

In [33]:
d_eq_c = Equals(d, c)

d_eq_c: d = c

In [34]:
d_eq_z = Equals(d, z)

d_eq_z: d = z

In [35]:
a_eq_z = Equals(a, z).prove(assumptions=[a_eq_b, c_eq_b, d_eq_c, d_eq_z])

a_eq_z: {a = b , c = b , d = c , d = z} |= a = z

In [36]:
a_eq_z.proof()

step type	requirements	statement
0	specialization	5, 1, 2	{a = b , c = b , d = c , d = z} |= a = z
	{x -> a}, {y -> d}, {z -> z}
1	specialization	5, 3, 4	{a = b , c = b , d = c} |= a = d
	{x -> a}, {y -> c}, {z -> d}
2	assumption		{d = z} |= d = z
3	specialization	5, 6, 7	{a = b , c = b} |= a = c
	{x -> a}, {y -> b}, {z -> c}
4	specialization	9, 8	{d = c} |= c = d
	{x -> d}, {y -> c}
5	axiom		|= forall_{x, y, z | x = y , y = z} (x = z)
	proveit.logic.equality.equalsTransitivity
6	assumption		{a = b} |= a = b
7	specialization	9, 10	{c = b} |= b = c
	{x -> c}, {y -> b}
8	assumption		{d = c} |= d = c
9	theorem		|= forall_{x, y | x = y} (y = x)
	proveit.logic.equality.equalsReversal
10	assumption		{c = b} |= c = b

The `applyTransivity` method applies the transitivity relation explicity. 

In [37]:
a_eq_b.prove([a_eq_b]).applyTransitivity(c_eq_b, assumptions=[c_eq_b])

{c = b , a = b} |= a = c

## Evaluation

An evaluation is an equality in which the right hand side is an irreducible value (specifically, an instance of `proveit.logic.irreducible_value.IrreducibleValue`, aliased as `proveit.logic.IrreducibleValue`).  An irreducible value represents a mathematical object in its simplest form.  It cannot be reduced.  The evaluation of an `IrreducibleValue` is itself.  When an evaluation is proven, the associated **KnownTruth** is stored for future reference in `Equals.evaluations` for making other evaluations.

`TRUE` ($\top$) and `FALSE` ($\bot$) are both `IrreducibleValue`s:

In [38]:
from proveit.logic import TRUE, FALSE, IrreducibleValue

In [39]:
isinstance(TRUE, IrreducibleValue)

True

In [40]:
isinstance(FALSE, IrreducibleValue)

True

In [41]:
TRUE.evaluation()

|= TRUE = TRUE

An `IrreducibleValue` should implement the `evalEquality` and `notEqual` methods to prove equality relations with other `IrreducibleValue`s as appropriate.

In [42]:
Equals(Equals(FALSE, TRUE), FALSE).deriveContradiction([Equals(FALSE, TRUE)])

{FALSE = TRUE} |= FALSE

In [43]:
TRUE.evalEquality(TRUE)

|= (TRUE = TRUE) = TRUE

In [44]:
TRUE.evalEquality(FALSE)

|= (TRUE = FALSE) = FALSE

In [45]:
TRUE.notEqual(FALSE)

|= TRUE != FALSE

A proven expression will evaluate to `TRUE`.

In [46]:
a_eq_T = a.evaluation([a])

a_eq_T: {a} |= a = TRUE

In [47]:
a_eq_T.proof()

step type	requirements	statement
0	specialization	1, 2	{a} |= a = TRUE
	A -> a
1	axiom		|= forall_{A | A} (A = TRUE)
	proveit.logic.boolean.eqTrueIntro
2	assumption		{a} |= a

A disproven expression will evaluate to `FALSE`.

In [48]:
a_eq_F = a.evaluation([Not(a)])

a_eq_F: {[not](a)} |= a = FALSE

In [49]:
a_eq_F.proof()

step type	requirements	statement
0	specialization	1, 2	{[not](a)} |= a = FALSE
	A -> a
1	axiom		|= forall_{A | [not](A)} (A = FALSE)
	proveit.logic.boolean.negation.negationElim
2	assumption		{[not](a)} |= [not](a)

If the expression to be evaluated is known to be equal to an expression that has already been evaluated, transitivity will automatically be applied.

In [50]:
b_eq_F = b.evaluation([Equals(b, a), Not(a)])

b_eq_F: {b = a , [not](a)} |= b = FALSE

In [51]:
b_eq_F.proof()

step type	requirements	statement
0	specialization	1, 2, 3	{b = a , [not](a)} |= b = FALSE
	{x -> b}, {y -> a}, {z -> FALSE}
1	axiom		|= forall_{x, y, z | x = y , y = z} (x = z)
	proveit.logic.equality.equalsTransitivity
2	assumption		{b = a} |= b = a
3	specialization	4, 5	{[not](a)} |= a = FALSE
	A -> a
4	axiom		|= forall_{A | [not](A)} (A = FALSE)
	proveit.logic.boolean.negation.negationElim
5	assumption		{[not](a)} |= [not](a)

When evaluating an expression, an evaluation of the operands will be attempted.  The operation class is responsible for overridding the `evaluate` method in order to properly treat the operation applied to irreducible values or generate a more efficient proof as appropriate.

In [52]:
from proveit.logic import Or, Not, inBool

In [53]:
nested_eval = Or(a, Not(a)).evaluation([a])

nested_eval: {a} |= (a or [not](a)) = TRUE

In [54]:
nested_eval.proof()

step type	requirements	statement
0	specialization	3, 1, 2	{a} |= (a or [not](a)) = TRUE
	{x -> (a or [not](a))}, {y -> (TRUE or FALSE)}, {z -> TRUE}
1	specialization	3, 4, 5	{a} |= (a or [not](a)) = (TRUE or FALSE)
	{x -> (a or [not](a))}, {y -> (TRUE or [not](TRUE))}, {z -> (TRUE or FALSE)}
2	axiom		|= (TRUE or FALSE) = TRUE
	proveit.logic.boolean.disjunction.orTF
3	axiom		|= forall_{x, y, z | x = y , y = z} (x = z)
	proveit.logic.equality.equalsTransitivity
4	specialization	7, 6	{a} |= (a or [not](a)) = (TRUE or [not](TRUE))
	{f -> [_x_ -> (_x_ or [not](_x_))]}, {x -> a}, {y -> TRUE}
5	specialization	7, 8	|= (TRUE or [not](TRUE)) = (TRUE or FALSE)
	{f -> [_x_ -> (TRUE or _x_)]}, {x -> [not](TRUE)}, {y -> FALSE}
6	specialization	9, 10	{a} |= a = TRUE
	A -> a
7	axiom		|= forall_{f, x, y | x = y} (f(x) = f(y))
	proveit.logic.equality.substitution
8	theorem		|= [not](TRUE) = FALSE
	proveit.logic.boolean.negation.notT
9	axiom		|= forall_{A | A} (A = TRUE)
	proveit.logic.boolean.eqTrueIntro
10	assumption		{a} |= a

## Boolean equality

Equality with `TRUE` ($\top$) or `FALSE` ($\bot$) has special logical consequences.  The `Equals` object has automation capabilities to treat these special kinds of equations.

In [55]:
TRUE

TRUE

In [56]:
FALSE

FALSE

Proofs via boolean equality are automatic via `Equals.deduceSideEffects`:

In [57]:
a.prove([Equals(a, TRUE)])

{a = TRUE} |= a

In [58]:
a.prove([Equals(a, TRUE)]).proof()

step type	requirements	statement
0	specialization	1, 2	{a = TRUE} |= a
	A -> a
1	axiom		|= forall_{A | A = TRUE} A
	proveit.logic.boolean.eqTrueElim
2	assumption		{a = TRUE} |= a = TRUE

In [59]:
Not(b).prove([Equals(b, FALSE)])

{b = FALSE} |= [not](b)

In [60]:
Not(b).prove([Equals(b, FALSE)]).proof()

step type	requirements	statement
0	specialization	1, 2	{b = FALSE} |= [not](b)
	A -> b
1	axiom		|= forall_{A | A = FALSE} [not](A)
	proveit.logic.boolean.negation.negationIntro
2	assumption		{b = FALSE} |= b = FALSE

Going the other direction, boolean equalities are proven automatically via `Equals.conclude`:

In [61]:
Equals(x, TRUE).prove([x])

{x} |= x = TRUE

In [62]:
Equals(TRUE, y).prove([y])

{y} |= TRUE = y

In [63]:
Equals(c, FALSE).prove([Not(c)])

{[not](c)} |= c = FALSE

In [64]:
Equals(FALSE, c).prove([Not(c)])

{[not](c)} |= FALSE = c

When something is equal to `FALSE` and known to be true, there is a contradiction.  That is, `FALSE` is a consequence.  The `deriveContradiction` method of `Equals` can be used to prove such a contradiction:

In [65]:
contradiction = Equals(a, FALSE).deriveContradiction([a, Equals(a, FALSE)])

contradiction: {a , a = FALSE} |= FALSE

In [66]:
contradiction.proof()

step type	requirements	statement
0	specialization	1, 2, 3	{a , a = FALSE} |= FALSE
	A -> a
1	theorem		|= forall_{A | A , A = FALSE} FALSE
	proveit.logic.equality.contradictionViaFalsification
2	assumption		{a} |= a
3	assumption		{a = FALSE} |= a = FALSE

Contradictions are useful in making contradiction proofs (reductio ad absurdum).  The `affirmViaContradiction` and `denyViaContradiction` methods are useful in making such a proof.  They both use `deriveContradiction`.  For example:

In [67]:
from proveit.logic import Implies, inBool
not_a__truth = Equals(a, FALSE).affirmViaContradiction(Not(a), [Implies(a, Equals(a, FALSE)), inBool(a)])

not_a__truth: {a => (a = FALSE) , a in BOOLEANS} |= [not](a)

In [68]:
not_a__truth.proof().numSteps()

13

It is more efficient to use `denyViaContradiction` when proving the negation of something.  Here we prove the same as above but in fewer steps.

In [69]:
not_b__truth = Equals(b, FALSE).denyViaContradiction(b, [Implies(b, Equals(b, FALSE)), inBool(b)])

not_b__truth: {b => (b = FALSE) , b in BOOLEANS} |= [not](b)

In [70]:
not_b__truth.proof().numSteps()

9

In [71]:
not_b__truth.proof()

step type	requirements	statement
0	specialization	1, 2, 3	{b => (b = FALSE) , b in BOOLEANS} |= [not](b)
	A -> b
1	theorem		|= forall_{A in BOOLEANS | A => FALSE} [not](A)
	proveit.logic.boolean.implication.denyViaContradiction
2	assumption		{b in BOOLEANS} |= b in BOOLEANS
3	hypothetical reasoning	4	{b => (b = FALSE)} |= b => FALSE
4	specialization	5, 8, 6	{b => (b = FALSE) , b} |= FALSE
	A -> b
5	theorem		|= forall_{A | A , A = FALSE} FALSE
	proveit.logic.equality.contradictionViaFalsification
6	modus ponens	7, 8	{b => (b = FALSE) , b} |= b = FALSE
7	assumption		{b => (b = FALSE)} |= b => (b = FALSE)
8	assumption		{b} |= b

## Equality and sets

All equality expressions are in the boolean set by the `equalityInBool` axiom.  That is, given any two mathematical objects, they are either equal or not (even nonsense is either the same nonsense or different nonsense).

In [72]:
eq_in_bool = Equals(a, b).deduceInBool()

eq_in_bool: |= (a = b) in BOOLEANS

In [73]:
eq_in_bool.proof()

step type	requirements	statement
0	specialization	1	|= (a = b) in BOOLEANS
	{x -> a}, {y -> b}
1	axiom		|= forall_{x, y} ((x = y) in BOOLEANS)
	proveit.logic.equality.equalityInBool

A singleton is a set with one element.  If $x=c$, then $x$ is in the singleton set of $\{c\}$:

In [74]:
in_singleton_truth = Equals(x, c).deriveIsInSingleton([Equals(x, c)])

in_singleton_truth: {x = c} |= x in {c}

In [75]:
in_singleton_truth.proof()

step type	requirements	statement
0	specialization	1, 2	{x = c} |= x in {c}
	{x -> x}, {y -> c}
1	theorem		|= forall_{x, y | x = y} (x in {y})
	proveit.logic.set_theory.enumeration.foldSingleton
2	assumption		{x = c} |= x = c

## Not equals

The `NotEquals` operation is a more way of expressing that two mathematical objects are not equal to each other.

In [76]:
from proveit.logic import NotEquals

In [77]:
a_neq_b = NotEquals(a, b)

a_neq_b: a != b

In [78]:
a_neq_b.definition()

|= (a != b) = [not](a = b)

In [79]:
a_neq_b.definition().proof()

step type	requirements	statement
0	specialization	1	|= (a != b) = [not](a = b)
	{x -> a}, {y -> b}
1	axiom		|= forall_{x, y} ((x != y) = [not](x = y))
	proveit.logic.equality.notEqualsDef

From $\lnot (a = b)$ one can derive $a \neq b$ and vice-versa, folding and unfolding the `NotEquals`.

In [80]:
a_neq_b.prove([Not(a_eq_b)])

{[not](a = b)} |= a != b

In [81]:
a_neq_b.prove([Not(a_eq_b)]).proof()

step type	requirements	statement
0	specialization	1, 2	{[not](a = b)} |= a != b
	{x -> a}, {y -> b}
1	theorem		|= forall_{x, y | [not](x = y)} (x != y)
	proveit.logic.equality.foldNotEquals
2	assumption		{[not](a = b)} |= [not](a = b)

In [82]:
Not(a_eq_b).prove([a_neq_b])

{a != b} |= [not](a = b)

In [83]:
Not(a_eq_b).prove([a_neq_b]).proof()

step type	requirements	statement
0	specialization	1, 2	{a != b} |= [not](a = b)
	{x -> a}, {y -> b}
1	theorem		|= forall_{x, y | x != y} [not](x = y)
	proveit.logic.equality.unfoldNotEquals
2	assumption		{a != b} |= a != b

`NotEquals` also has a symmetry property which can be applied directly.  It can be proven  through automation.

In [84]:
NotEquals(b, a).prove([a_neq_b])

{a != b} |= b != a

Or explicitly via `deriveReversed`

In [85]:
a_neq_b.prove([a_neq_b]).deriveReversed().proof()

step type	requirements	statement
0	specialization	1, 2	{a != b} |= b != a
	{x -> a}, {y -> b}
1	theorem		|= forall_{x, y | x != y} (y != x)
	proveit.logic.equality.notEqualsSymmetry
2	assumption		{a != b} |= a != b

If two objects are both equal and not equal, there is a contradiction.

In [86]:
neq_contradiction = a_neq_b.deriveContradiction([a_neq_b, a_eq_b])

neq_contradiction: {a != b , a = b} |= FALSE

In [87]:
neq_contradiction.proof()

step type	requirements	statement
0	specialization	1, 2, 3	{a != b , a = b} |= FALSE
	{x -> a}, {y -> b}
1	theorem		|= forall_{x, y | x = y , x != y} FALSE
	proveit.logic.equality.notEqualsContradiction
2	assumption		{a = b} |= a = b
3	assumption		{a != b} |= a != b

In [88]:
a_neq_a = NotEquals(a, a)

a_neq_a: a != a

In [89]:
b_from_contradiction = a_neq_a.affirmViaContradiction(b, [Implies(Not(b), a_neq_a), inBool(b)])

b_from_contradiction: {[not](b) => (a != a) , b in BOOLEANS} |= b

In [90]:
b_from_contradiction.proof()

step type	requirements	statement
0	specialization	1, 2, 3	{[not](b) => (a != a) , b in BOOLEANS} |= b
	A -> b
1	theorem		|= forall_{A in BOOLEANS | [not](A) => FALSE} A
	proveit.logic.boolean.implication.affirmViaContradiction
2	assumption		{b in BOOLEANS} |= b in BOOLEANS
3	hypothetical reasoning	4	{[not](b) => (a != a)} |= [not](b) => FALSE
4	specialization	5, 6, 7	{[not](b) => (a != a) , [not](b)} |= FALSE
	{x -> a}, {y -> a}
5	theorem		|= forall_{x, y | x = y , x != y} FALSE
	proveit.logic.equality.notEqualsContradiction
6	specialization	8	|= a = a
	x -> a
7	modus ponens	9, 10	{[not](b) => (a != a) , [not](b)} |= a != a
8	axiom		|= forall_{x} (x = x)
	proveit.logic.equality.equalsReflexivity
9	assumption		{[not](b) => (a != a)} |= [not](b) => (a != a)
10	assumption		{[not](b)} |= [not](b)

In [91]:
%end demonstrations